In [1]:
from models import *
from pytorch_robust_pca import *
from utils import *

In [2]:
import numpy as np

# generate low rank synthetic data
N = 100
num_groups = 3
num_values_per_group = 40
p_missing = 0.2

Ds = []
for k in range(num_groups):
    d = np.ones((N, num_values_per_group)) * (k + 1) * 10
    Ds.append(d)

D = np.hstack(Ds)

# decimate 20% of data
n1, n2 = D.shape
S = np.random.rand(n1, n2)
D[S < 0.2] = 0

D = torch.FloatTensor(D).requires_grad_(False)

In [3]:
model = RobustPCANeuralNet(input_tensor=D)

The settings are ...
Lambda: tensor(0.0913)
Inverse mu: tensor(0.9100)


In [4]:
opt = torch.optim.Adam(model.parameters(), lr=1e-2)
for i in range(10000):
    opt.zero_grad()
    L = model(D)
    l = model.loss(D, L)
    l.backward()
    if torch.sum(torch.isnan(model.S.grad)) > 0: opt.zero_grad(); break
    opt.step()
    if i%1000==0: print(l.item())
        
print(model.S.grad)

9664.828125
3587.5087890625
1433.458984375
1336.272705078125
1241.936767578125
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [5]:
((D-(L+model.S))**2).mean()

tensor(0., grad_fn=<MeanBackward0>)

In [6]:
D-(L+model.S)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<SubBackward0>)

In [7]:
print(torch.linalg.matrix_rank(L).item())
print(max(L.shape))

34
120


In [8]:
100*(34/120)

28.333333333333332